# Projeto Final 
## Master em Jornalismo de Dados - Insper

Trabalho apresentado na conclusão do Master em Jornalismo de Dados do Insper em 2023. O projeto foi ministrado pelo professor Eduardo Vicente Gonçalves.  

### O Projeto

A ideia do trabalho surgiu da minha cobertura como setorista de companhias aéreas para o Valor Econômico. Uma dos temas mais presentes no noticiário é o preço da passagem aérea. As tarifas dispararam nos últimos anos por causa dos impactos da pandemia e do efeito da invasão da Rússia na Ucrânia sobre o petróleo. 

A Anac faz um monitoramento intensivo das tarifas, que é disponibilizado em um dashboard interativo bastante eficiente (https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/mercado-do-transporte-aereo), na aba Tarifas Aéreas Domésticas e Tarifas Aéreas Internacionais).

O problema é que eu gostaria de fazer os meus próprios cruzamentos com os microdados e também trabalhar com novas visualizações. 

#### Os Dados

A Anac recolhe das companhias aéreas comerciais uma série de dados mensalmente para calcular a oferta e demanda por transporte, assim como a evolução das tarifas. No caso das tarifas, os dados representam uma fatia de cerca de 35% do total de passageiros transportados (a depender do mês). Esse recorde ocorre, conforme explica a Anac, porque os dados abrangem todas as passagens vendidas ao “público adulto em geral”, excluindo aquelas adquiridas com descontos restritos a grupos específicos, programas de milhagem, entre outras condições. 

As variáveis são diretas:
- ANO e MES: mês e ano da venda das passagens, independentemente da data do voo;
- EMPRESA: (empresa que vendeu a passagem);
- ORIGEM e DESTINO;
- TARIFA (em reais);
- ASSENTOS: por exemplo, uma linha com venda da Azul no mês de janeiro de 2023 por 500 reais com um número de assentos de 9 signifca que, naquele mês a empresa vendeu 9 assentos por 500 reais naquela rota. 

Os microdados estão disponíveis no seguinte link: https://sas.anac.gov.br/sas/downloads/view/frmDownload.aspx

### Ferramentas - Spark

A base completa - janeiro de 2002 a dezembro de 2022 - tem mais de 55 milhões de registros distribuídos em 250 arquivos. Desta forma, abrir o arquivo no Pandas não seria viável na memória do computador. 

Para cruzar os dados, resolvi usar o Pyspark - interface Python para Spark e é um projeto Apache de plataforma cruzada de código aberto. 

### Docker
O Spark é bastante simples de ser iniciado via Docker, método escolhido para este projeto. 
Basta clonar o repositório. Na pasta principal, digite `docker-compose up --build -d` - com isso o Docker vai construir os containeres, iniciar o sistema e manter o terminal livre (detached, ou -d). Ele vai iniciar o Spark, assim como um Worker para processar os dados. 

### Jupyterhttp://localhost:8888/lab
O Docker vai criar também um Jupyter, que pode ser acessado pelo seguinte endereço: http://localhost:8888/
Aqui vai ser possível navegar pela pasta Anac e encontrar os notebooks, assim como os arquivos. 

# Limpeza dos Dados

O primeiro desafio encontrado foi a limpeza dos dados. 
Minha máquina acabou não suportando todos os processamentos necessários no dado. Entre eles, preciso transformar variáveis, ajustar alguns CSVs com formatação e header errados. O principal desafio está na correção monetária: por padrão a Anac atualiza todos os bilhetes comercializados desde 2002 pelo IPCA acumulado no mês da última divulgação. 

In [1]:
import numpy as np
import pandas as pd
import glob


# Ajustando dados e trazendo tudo a valor presente
Um dos desafios mais legais foi pensar em uma função para ajustar os valores pelo IPCA.
Por se tratar de uma função que seria executada literalmente milhares de vezes, ela teria de ser rápida e bastante eficiente. 
A primeira versão corrigiu todos os dados de uma tabela de cerca de 330 mil entradas em cerca de 20 minutos. A segunda, conseguiu fazer o mesmo trabalho em cerca de 1,3 minuto - um salto bastante significativo para uma base com 250 CSVs, sendo cada um variando entre 200 mil e 500 mil entradas cada. 

In [ ]:
#Função criada para ajustar os valores individualmente. Ela foi testada com base na Calculadora do BC. 
from udfs import ipca

In [ ]:
%%time
def update_db(month_ipca):
    """
    Função que pega os CSVs individualmente, limpa os dados e corrige a tarifa pelo IPCA do parâmetro month_ipca. 
    A data precisa ser no formato "MM/YYYY"
    """
    
    files = glob.glob("anac.nosync/*.CSV")
    
    for file in files:
        print(f"\r>>Processando tabela {file}", end="")
        df = pd.read_csv(file, sep=";", encoding='latin1')
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        df.columns = ["ANO","MES","EMPRESA","ORIGEM","DESTINO","TARIFA","ASSENTOS"]
        df.TARIFA = df.TARIFA.str.replace(",", ".").astype(float).copy()

        if file == files[-1]:    
            df['REAL'] = df['TARIFA'].copy()
        else:
            df['REAL'] = df.apply(lambda row: ipca(row["TARIFA"], f"{row['MES']}/{row['ANO']}", month_ipca)[0], axis=1)

        df.to_csv(f"anac.nosync/real/{file.split('/')[1]}.csv", index=False, sep=";")

#update_db("12/2022")

Processando tabela anac.nosync/201305.CSV

# Começando com o Spark
Lider com o Spark foi outro desafio.
Ele demnanda uma série de configurações completas e ajustar tudo ao meu computador foi trabalhoso. Mas consegui encontrar uma saída (com este Docker container, inspirado em um projeto do github (link nas referências). 

In [73]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as f

conf = SparkConf()
# conf.set("spark.eventLog.enabled", False)
# conf.set("spark.eventLog.dir", "/opt/spark/spark-events")
# conf.set("spark.history.fs.logDirectory", "/opt/spark/spark-events")
# conf.set("spark.executor.memory", "1024m")
# conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# conf.set("spark.sparkContext.setLogLevel", "ERROR")

spark = SparkSession.\
        builder.\
        appName("pyspark-anac").\
        master("spark://spark-master:7077").\
        config(conf=conf).\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/11 20:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [74]:
spark
# sc = spark.sparkContext
# sc.addPyFile("/opt/notebooks/udfs.py")

## Construindo o Schema
Parte importante para definir os tipos de arquivos. Uma boa definição ajuda o programa a rodar significativamente mais rápido. 

In [75]:
from pyspark.sql.types import IntegerType, StringType, FloatType

schema = StructType(
    [
        StructField('ANO', IntegerType(), True), 
        StructField('MES', IntegerType(), True), 
        StructField('EMPRESA', StringType(), True), 
        StructField('ORIGEM', StringType(), True), 
        StructField('DESTINO', StringType(), True), 
        StructField('TARIFA', FloatType(), True), 
        StructField('ASSENTOS', IntegerType(), True),
        StructField('REAL', FloatType(), True),
    ]
)

Agora vamos ler todos os arquivos editados de uma só vez. Isso é possível com o "*" antes do ".csv", que significa que o Spark irá ler todos os arquivos CSVs no diretório. Importante: todos precisam ter a mesma quantidade de colunas.

In [76]:
sdf = spark.read.csv('anac.nosync/real/*.csv', header=True, sep=";", schema=schema)

In [78]:
#Vamos contar a quantidade de linhas
sdf.count()

55291652

In [ ]:
#Uma breve análise exploratória dos dados
sdf.summary()

In [79]:
#Vendo apenas as primeiras três linhas da tabela. 
sdf.limit(3).show()

+----+---+-------+------+-------+------+--------+------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO|TARIFA|ASSENTOS|  REAL|
+----+---+-------+------+-------+------+--------+------+
|2021| 10|    AZU|  SBAC|   SBAR| 565.9|       1|616.34|
|2021| 10|    AZU|  SBAC|   SBBR| 409.9|       1|446.44|
|2021| 10|    AZU|  SBAC|   SBBV| 748.9|       4|815.66|
+----+---+-------+------+-------+------+--------+------+



# Ajustar todos os valanac.nosync/ pela inflação de Dezembro de 2022

Nao funciona aqui por questão de processamento. Por isso, exacutamos via pandas e editando e corrigindo os arquivos individualmente. 

```
def valor_real(value, date_start):
    return ipca(value, date_start, "12/2022")[0]

vreal = f.udf(valor_real, DoubleType())    # registering as udf here


sdf = sdf.withColumn(
    'REAL',
    vreal(
        f.col("TARIFA"), 
        f.concat(f.col("MES"), f.lit("/"), f.col("ANO")),
    )
)

sdf = sdf.select("ANO", "MES", "ASSENTOS", "REAL")
````


# Dado final

## Agrupamento por ano e mês
Primeiro vamos agrupar os dados pelas informações que a gente quer. No caso vamos pegar as tarifas médias por mês. 

In [80]:
data = sdf.groupby("ANO", "MES").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('REAL')),
    f.collect_list(f.col('ASSENTOS')),
)

In [81]:
#precisamos de uma média ponderada para calcular a real representatividade das tarifas considerando o número de assentos. 
def get_average(values, weights):
    return float(np.average(values, weights=weights)) 

waverage = f.udf(get_average, DoubleType())    # registering as udf here

data = data.withColumn(
    'Weighted_average',
    waverage(
        f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)")
    )
)

In [82]:
data.cache() #salvando a base no cache para usar depois. 

DataFrame[ANO: int, MES: int, TOTAL: bigint, collect_list(REAL): array<float>, collect_list(ASSENTOS): array<int>, Weighted_average: double]

In [83]:
data.limit(3).show()

+----+---+-------+--------------------+----------------------+-----------------+
| ANO|MES|  TOTAL|  collect_list(REAL)|collect_list(ASSENTOS)| Weighted_average|
+----+---+-------+--------------------+----------------------+-----------------+
|2022| 10|2450901|[1067.1, 660.59, ...|  [1, 2, 3, 26, 8, ...|648.7533683010134|
|2012| 10|4443441|[553.07, 1109.43,...|  [2, 1, 1, 2, 1, 7...|567.5236231638822|
|2007|  6|1366528|[459.67, 1140.66,...|  [103, 10, 56, 7, ...|681.8399205044295|
+----+---+-------+--------------------+----------------------+-----------------+



In [ ]:
#Jogando para CSV para futura consulta sem precisar rodar de novo
data.select("ANO", "MES", "TOTAL", "Weighted_average")\
    .repartition(1)\
    .write.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .save("tarifa_real.csv")

### Análise - Tarifas mensais. 
- Criar gráficos mostrando a evolução das tarifas

In [88]:
df_final = data.select("ANO", "MES", "TOTAL", "Weighted_average").toPandas()
df_final.sort_values(["ANO", "MES"], ascending=False)

,ANO,MES,TOTAL,Weighted_average
243,2022,12,1826543,654.853020
92,2022,11,2456623,641.613440
0,2022,10,2450901,648.753368
220,2022,9,2124662,707.491415
248,2022,8,2080709,694.486447
...,...,...,...,...
56,2002,5,1139840,726.007524
185,2002,4,1158974,743.106776
11,2002,3,1226532,787.954982
158,2002,2,1150345,843.791893


## Tarifa por companhia aérea
- Criar gráfico por companhia aérea

In [131]:
df_final[df_final["MES"] == 10].sort_values("ANO", ascending=False)

,ANO,MES,TOTAL,Weighted_average
0,2022,10,2450901,648.753368
35,2021,10,3402952,611.229469
195,2020,10,2106124,510.975563
7,2019,10,3387866,566.608174
19,2018,10,3242480,532.268112
18,2017,10,3556089,522.318945
118,2016,10,3143825,544.979843
134,2015,10,3737626,553.915428
48,2014,10,4571031,530.376370
60,2013,10,4527674,608.515879


# Fechando aqui

In [1]:
import pandas as pd
import numpy as np

In [119]:
from monetary_correction import MonetaryCorrection

In [4]:
corretor = ipca

# Outubro

In [161]:
outubro = pd.read_csv("anac.nosync/202210.CSV", sep=";", encoding="latin1")
outubro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
outubro.TARIFA = outubro.TARIFA.str.replace(",", ".").astype(float)

In [162]:
outubro['REAL'] = outubro.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

KeyboardInterrupt: 

In [ ]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=outubro.loc[x.index, "ASSENTOS"])

media_outubro = outubro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("TARIFA", wm))
media_outubro

In [99]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=outubro.loc[x.index, "ASSENTOS"])

media_outubro = outubro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("TARIFA", wm))
media_outubro

,,price_weighted_mean
ANO,MES,
2022,10,638.356865


In [175]:
# Trabalhando novembro

In [179]:
novembro_2022 = pd.read_csv("anac.nosync/real/201910.CSV", sep=";", encoding="latin1")
# novembro_2022.TARIFA = novembro_2022.TARIFA.str.replace(",", ".").astype(float)
# novembro_2022['REAL'] = novembro_2022.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

In [180]:
novembro_2022

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,REAL
0,2019,10,AZU,SBAC,SBAR,181.9,1,225.26
1,2019,10,AZU,SBAC,SBAR,531.9,6,658.70
2,2019,10,AZU,SBAC,SBBV,607.9,2,752.81
3,2019,10,AZU,SBAC,SBCF,247.9,5,306.99
4,2019,10,AZU,SBAC,SBCF,272.9,2,337.95
...,...,...,...,...,...,...,...,...
374032,2019,10,TAM,SWGN,SBPJ,509.0,1,630.34
374033,2019,10,TAM,SWGN,SBPJ,554.0,2,686.06
374034,2019,10,TAM,SWGN,SBPJ,574.0,2,710.83
374035,2019,10,TAM,SWGN,SBPJ,1499.5,1,1856.95


In [101]:
outubro.head(10)

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,REAL
0,2022,10,ABJ,SBSV,SIRI,1050.0,1,1067.100657
1,2022,10,ABJ,SBSV,SIRI,650.0,2,660.586121
2,2022,10,ABJ,SBSV,SIRI,690.0,3,701.237575
3,2022,10,ABJ,SBSV,SNCL,1150.0,26,1168.729291
4,2022,10,ABJ,SBSV,SNCL,1190.0,8,1209.380745
5,2022,10,ABJ,SBSV,SNCL,1400.0,8,1422.800876
6,2022,10,ABJ,SBSV,SNCL,490.0,10,497.980307
7,2022,10,ABJ,SBSV,SNCL,550.0,33,558.957487
8,2022,10,ABJ,SBSV,SNCL,750.0,40,762.214755
9,2022,10,ABJ,SBSV,SNCL,790.0,15,802.866209


In [171]:
class MonetaryCorrection:
    def __init__(self):

        ipca_data = pd.read_csv("https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=csv", sep=";")
        ipca_data["valor"] = ipca_data.valor.str.replace(",", ".").astype(float)
        ipca_data["taxa"] = ipca_data["valor"]/100 + 1     
        ipca_data["data"] = ipca_data["data"].str[3:]
        ipca_data['correction_index'] = ipca_data.taxa.cumprod()
        self.ipca_data = ipca_data

        self.ipca_data_dict = { data: value for data,value in ipca_data[["data", "correction_index"]].values }


    def ipca(self, value, date_start, date_end):
        """
        >>> corretor.ipca(100, "07/2019", "02/2023")
        (125.87, 25.86683)
        
        >>> corretor.ipca(231.24, "01/2002", "02/2023")
        (837.26, 262.07534)
        
        >>> corretor.ipca(231.24, "1/2002", "2/2023")
        (837.26, 262.07534)
               
        """

        date_start_valid = (pd.to_datetime(date_start, format="%m/%Y") - pd.DateOffset(months=1)).strftime('%m/%Y')
        correction_index_start = self.ipca_data_dict[date_start_valid]
        correction_index_end = self.ipca_data_dict[("0" + date_end)[-7:]]

        correction_index_in_time = (correction_index_end / correction_index_start)

        return round(value * correction_index_in_time, 2), round((correction_index_in_time - 1) * 100, 5)
    
 
corretor = MonetaryCorrection().ipca

In [172]:
outubro_2019 = pd.read_csv("anac.nosync/201910.CSV", sep=";", encoding="latin1")
outubro_2019.TARIFA = outubro_2019.TARIFA.str.replace(",", ".").astype(float)
outubro_2019['REAL'] = outubro_2019.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

In [173]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=outubro_2019.loc[x.index, "ASSENTOS"])

media_outubro_2019 = outubro_2019.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))
media_outubro_2019

,,price_weighted_mean
ANO,MES,
2019,10,566.608174


In [44]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=outubro_2019.loc[x.index, "ASSENTOS"])

media_outubro_2019 = outubro_2019.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))
media_outubro_2019

,,price_weighted_mean
ANO,MES,
2019,10,560.817476


In [51]:
(float(media_outubro['price_weighted_mean']) - float(media_outubro_2019['price_weighted_mean'])) / float(media_outubro_2019['price_weighted_mean']) *100

13.82613636966195

# Dezembro 22 - Dezembro 2019

In [58]:
dezembro = pd.read_csv("anac.nosync/202212.CSV", sep=";", encoding="latin1")
# dezembro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
dezembro.TARIFA = dezembro.TARIFA.str.replace(",", ".").astype(float)


In [67]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

media_dezembro = dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("TARIFA", wm))
media_dezembro

,,price_weighted_mean
ANO,MES,
2022,12,650.817921


In [63]:
dezembro_2019 = pd.read_csv("anac.nosync/201912.CSV", sep=";", encoding="latin1")
dezembro_2019.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
dezembro_2019.TARIFA = dezembro_2019.TARIFA.str.replace(",", ".").astype(float)

In [65]:
dezembro_2019['REAL'] = dezembro_2019.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "10/2022")[0], axis=1) 

In [66]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro_2019.loc[x.index, "ASSENTOS"])

media_dezembro_2019 = dezembro_2019.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))
media_dezembro_2019

,,price_weighted_mean
ANO,MES,
2019,12,531.823898


In [68]:
(float(media_dezembro['price_weighted_mean']) - float(media_dezembro_2019['price_weighted_mean'])) / float(media_dezembro_2019['price_weighted_mean']) *100

22.374703943441464

In [11]:
dezembro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']

In [12]:
dezembro.TARIFA = dezembro.TARIFA.str.replace(",", ".").astype(float)

In [28]:
len(dezembro)

360948

In [29]:
%%time
dezembro['REAL'] = dezembro.apply(lambda row: corretor(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "07/2022")[0], axis=1) 

CPU times: user 8min 58s, sys: 5.61 s, total: 9min 4s
Wall time: 9min 16s


In [30]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,486.936127


In [27]:
# Ver se vai dar o mesmo resultado em dezembro.
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,491.655068


In [23]:
# Ver se vai dar o mesmo resultado em dezembro.
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,491.654549


In [51]:
%%time
dezembro['REAL'] = dezembro.apply(lambda row: corretor.ipca(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

CPU times: user 19min 31s, sys: 12.1 s, total: 19min 43s
Wall time: 20min 34s


In [53]:
dezembro

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,REAL
0,2022,12,ABJ,SBSV,SIRI,550.0,8,553.41
1,2022,12,ABJ,SBSV,SIRI,650.0,7,654.03
2,2022,12,ABJ,SBSV,SIRI,850.0,11,855.27
3,2022,12,ABJ,SBSV,SIRI,1050.0,7,1056.51
4,2022,12,ABJ,SBSV,SIRI,1250.0,3,1257.75
...,...,...,...,...,...,...,...,...
331370,2022,12,TAM,SWSI,SBVT,837.0,1,842.19
331371,2022,12,TAM,SWSI,SBVT,1395.0,1,1403.65
331372,2022,12,TAM,SWSI,SBVT,1813.9,1,1825.15
331373,2022,12,TAM,SWSI,SBVT,1866.0,1,1877.57


In [11]:
# %%time
# dezembro['REAL'] = dezembro.apply(lambda row: corretorOld.ipca(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

CPU times: user 22min 41s, sys: 15.4 s, total: 22min 56s
Wall time: 23min 44s


In [36]:

wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2022,12,654.853019


In [ ]:
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2020,12,491.654549


In [ ]:
def get_average(values, weights):
    return float(np.average(values, weights=weights))    # function created to calculate wt. median


In [116]:
df["real"] = df[1:].apply(lambda row: corretor.ipca(row["Weighted_average"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

In [117]:
df[df["MES"]==12]

,ANO,MES,Total,Weighted_average,real
244,2022,12,1826543,650.817925,NaN
115,2021,12,2421438,490.797236,522.98
110,2020,12,1868805,416.657935,491.66
185,2019,12,2585896,436.533702,537.32
197,2018,12,2498232,444.033223,564.44
125,2017,12,2745175,430.039663,568.77
147,2016,12,2839457,363.795988,494.65
21,2015,12,3286404,358.082682,520.90
52,2014,12,3915556,333.243424,535.55
78,2013,12,3790080,351.360833,601.69


# Referências
- Docker Pyspark: https://github.com/mrn-aglic/spark-standalone-cluster
- Calculo da Variação pelo IPCA: https://estudaradm.com.br/macroeconomia/calcular-inflacao-acumulada-em-12-meses-exemplos-e-exercicios/?unapproved=1687&moderation-hash=42b2b86a1784537b7b5a3c2fa55d719e#comment-1687 (OBS: obrigado especial ao criador desta publicação. Todos os blogs que eu li falando que iriam ensinar a deflacionar um número apenas usavam a calculadora do BC)